In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/data/Face_Test/Test/fear/168_m_f_f_b.jpg
/kaggle/input/dataset/data/Face_Test/Test/fear/168_m_f_f_a.jpg
/kaggle/input/dataset/data/Face_Test/Test/sadness/168_m_f_s_b.jpg
/kaggle/input/dataset/data/Face_Test/Test/sadness/168_m_f_s_a.jpg
/kaggle/input/dataset/data/Face_Test/Test/neutrality/168_m_f_n_a.jpg
/kaggle/input/dataset/data/Face_Test/Test/neutrality/168_m_f_n_b.jpg
/kaggle/input/dataset/data/Face_Test/Test/hapiness/168_m_f_h_b.jpg
/kaggle/input/dataset/data/Face_Test/Test/hapiness/168_m_f_h_a.jpg
/kaggle/input/dataset/data/Face_Test/Test/disgust/168_m_f_d_b.jpg
/kaggle/input/dataset/data/Face_Test/Test/disgust/168_m_f_d_a.jpg
/kaggle/input/dataset/data/Face_Test/Test/anger/168_m_f_a_a.jpg
/kaggle/input/dataset/data/Face_Test/Test/anger/168_m_f_a_b.jpg
/kaggle/input/dataset/data/Face_Train/Train/fear/004_o_m_f_a.jpg
/kaggle/input/dataset/data/Face_Train/Train/fear/079_o_f_f_a.jpg
/kaggle/input/dataset/data/Face_Train/Train/fear/066_y_m_f_b.jpg
/kaggle/input/d

In [2]:
import torch
import torchvision
import glob
import numpy
import cv2
import math
import matplotlib.pyplot as plt
from skimage import io
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize

In [3]:
#train and test data directory
data_dir = "/kaggle/input/dataset/data/Face_Train/Train"
test_data_dir = "/kaggle/input/dataset/data/Face_Test/Test"

#load the train and test data
dataset = ImageFolder(data_dir,transform = transforms.Compose([transforms.Resize((250,200)),transforms.ToTensor()]))
test_dataset = ImageFolder(test_data_dir,transforms.Compose([transforms.Resize((250,200)),transforms.ToTensor()]))

In [ ]:
print(len(dataset))
print(len(test_dataset))

In [ ]:
img, label = dataset[0]
print(img.shape,label)
print("Clases del dataset : \n",dataset.classes)

In [ ]:
def display_img(img,label):
    print(f"Label : {dataset.classes[label]}")
    plt.imshow(img.permute(1,2,0))

display_img(*dataset[1])

In [ ]:
batch_size = 256
#val_size = 4
#train_size = len(dataset) - val_size 
#train_data,val_data = random_split(dataset,[train_size,val_size])
#print(f"Tamano del train : {len(train_data)}")
#print(f"Tamano de la validacion : {len(val_data)}")

train_data=dataset
val_data=test_dataset

train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 2, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 2, pin_memory = True)

In [ ]:
def show_batch(dl):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(val_dl)

In [ ]:
def show_batch(dl):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(train_dl)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
num_classes = 6

class CNNAlex(CNN):
    def __init__(self,num_classes=6):
        super(CNNAlex, self).__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,6)
            
        )
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
class CNNIn(CNN):
    def __init__(self,num_clases=6):
        super(CNNIn,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.fc = nn.Linear(in_features=125 * 100 * 24, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.relu1(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.pool(output)
        output = self.conv3(output)
        output = self.relu3(output)
        output = self.conv4(output)
        output = self.relu4(output)
        output = output.view(-1, 125 * 100 * 24)
        output = self.fc(output)
        return output

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def testeo(model):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_dl:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
        print(correct)
        print(total)
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

@torch.no_grad()        

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, model, train_loader, val_loader, optimizer):
    
    history = []
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [ ]:
num_epochs = 50
learning_rate = 0.001
model = CNNIn(6)
opt_func = torch.optim.Adam(model.parameters(), lr = learning_rate)

history = fit(num_epochs, model, train_dl, val_dl, opt_func)

In [ ]:
testeo(model)

In [ ]:
def plot_grap(history):
    #accuracies = [x['val_acc'] for x in history]
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    #plt.plot(accuracies, '-x')
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Entrenamiento','Validacion'])
    plt.title('Grafico');

plot_grap(history)

In [ ]:
def plot_grap(history):
    accuracies = [x['val_acc'] for x in history]
    #train_losses = [x.get('train_loss') for x in history]
    #val_acc = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    #plt.plot(train_losses, '-bx')
    #plt.plot(val_acc, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.legend(['Acurracy'])
    plt.title('Grafico de accuracy');

plot_grap(history)

In [ ]:
@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

get_all_preds(model,val_dl)

In [ ]:
with torch.no_grad():
    #prediction_loader = torch.utils.data.DataLoader(train_dl, batch_size=10000)
    train_preds = get_all_preds(model, val_dl)

In [ ]:
print(train_preds.shape)
#print(test_dataset.targets)
#a=test_dataset.targets.unsqueeze(0)
#print(a)

In [ ]:
def get_num_correct(preds, labels):
    labels=torch.from_numpy(np.array(labels))
    return preds.argmax(dim=1).eq(labels).sum().item()

preds_correct = get_num_correct(train_preds, test_dataset.targets)
print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(test_dataset))

In [ ]:
temp=torch.from_numpy(np.array(test_dataset.targets))
stacked = torch.stack((temp,train_preds.argmax(dim=1)),dim=1)

In [ ]:
stacked[0].tolist()

In [ ]:
cmt = torch.zeros(10,10, dtype=torch.int64)
cmt

In [ ]:
for p in stacked:
    tl, pl = p.tolist()
    print(tl,pl)
    cmt[tl, pl] = cmt[tl, pl] + 1

cmt

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_dataset.targets, train_preds.argmax(dim=1))
print(type(cm))
cm

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, test_dataset.classes)

In [ ]:
num_epochs = 100
learning_rate = 1.0
model = CNNIn(6)
opt_func = torch.optim.Adadelta(model.parameters(), lr = learning_rate,rho=0.9,eps=1e-06,weight_decay=0)

history = fit(num_epochs, model, train_dl, val_dl, opt_func)

In [ ]:
def plot_grap(history):
    #accuracies = [x['val_acc'] for x in history]
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    #plt.plot(accuracies, '-x')
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Entrenamiento','Validacion'])
    plt.title('Grafico');

plot_grap(history)

In [ ]:
def plot_grap(history):
    accuracies = [x['val_acc'] for x in history]
    #train_losses = [x.get('train_loss') for x in history]
    #val_acc = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    #plt.plot(train_losses, '-bx')
    #plt.plot(val_acc, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.legend(['Acurracy'])
    plt.title('Grafico de accuracy');

plot_grap(history)

In [ ]:
num_epochs = 50
learning_rate = 0.01
model = CNNIn(6)
opt_func = torch.optim.Adagrad(model.parameters(), lr = learning_rate,lr_decay=0,weight_decay=0,
                               initial_accumulator_value=0,eps=1e-10)

history = fit(num_epochs, model, train_dl, val_dl, opt_func)

In [ ]:
def plot_grap(history):
    #accuracies = [x['val_acc'] for x in history]
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    #plt.plot(accuracies, '-x')
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Entrenamiento','Validacion'])
    plt.title('Grafico');

plot_grap(history)

In [ ]:
def plot_grap(history):
    accuracies = [x['val_acc'] for x in history]
    #train_losses = [x.get('train_loss') for x in history]
    #val_acc = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    #plt.plot(train_losses, '-bx')
    #plt.plot(val_acc, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.legend(['Acurracy'])
    plt.title('Grafico de accuracy');

plot_grap(history)